In [1]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np


In [2]:
# path to the Million Song Dataset subset (uncompressed)
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_subset_path='/home/chowder/Documents/AiLearning/codingChallenges/kaggle/competitions/MillionSongDataset/MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check

In [3]:
# path to the Million Song Dataset code
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_code_path='/home/chowder/Documents/MSongsDB'
assert os.path.isdir(msd_code_path),'wrong path' # sanity check
# we add some paths to python so we can import MSD code
# Ubuntu: you can change the environment variable PYTHONPATH
# in your .bashrc file so you do not have to type these lines
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )
# imports specific to the MSD
import hdf5_getters as GETTERS

In [4]:
# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

In [5]:
# we define this very useful function to iterate the files
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [6]:
# we can now easily count the number of files in the dataset
print('number of song files:',apply_to_all_files(msd_subset_data_path))

number of song files: 10000


In [7]:
# let's now get all artist names in a set(). One nice property:
# if we enter many times the same artist, only one will be kept.
all_artist_names = set()

# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()
    

In [8]:
# let's apply the previous function to all files
# we'll also measure how long it takes
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_artist_name)
t2 = time.time()
print('all artist names extracted in:',strtimedelta(t1,t2))

all artist names extracted in: 0:00:36.669028


In [9]:
print(len(all_artist_names))

4412


In [10]:
# let's see some of the content of 'all_artist_names'
print('found',len(all_artist_names),'unique artist names')
for k in range(5):
    print(list(all_artist_names)[k])

found 4412 unique artist names
b'Nouvelle'
b"Mart'n\xc3\xa1lia"
b'Ms. Dynamite / Kymani Marley'
b'Karen Peck & New River'
b'Energy'


In [14]:
# this is too long, and the work of listing artist names has already
# been done. Let's redo the same task using an SQLite database.
# We connect to the provided database: track_metadata.db
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
# we build the SQL query
q = "SELECT DISTINCT artist_name FROM songs"
# we query the database
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print('all artist names extracted (SQLite) in:',strtimedelta(t1,t2))
# we close the connection to the database
conn.close()
# let's see some of the content
for k in range(5):
    print(all_artist_names_sqlite[k][0])

all artist names extracted (SQLite) in: 0:00:00.005282
!!!
(hed) p.e.
089 Clique feat. Minnesota Snipe & Skinny Cueball
089 Clique feat. Prophet
1. Futurologischer Congress


In [19]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
# we build the SQL query
q = "SELECT * FROM songs LIMIT 6"
# we query the database
t1 = time.time()
res = conn.execute(q)
first_track_meta = res.fetchall()
t2 = time.time()
print('all artist names extracted (SQLite) in:',strtimedelta(t1,t2))
# we close the connection to the database
conn.close()
# let's see some of the content
for s in first_track_meta:
    print(s)

all artist names extracted (SQLite) in: 0:00:00.000413
('TRACCVZ128F4291A8A', 'Deep Sea Creature', 'SOVLGJY12A8C13FBED', 'Call of the Mastodon', 'ARMQHX71187B9890D3', 'bc5e2ad6-0a4a-4d90-b911-e9a7e6861727', 'Mastodon', 280.21506, 0.780461748777, 0.574274730517, 2001)
('TRACCMH128F428E4CD', 'No Quieras Marcharte', 'SOGDQZK12A8C13F37C', 'Adelante', 'AR2PT4M1187FB55B1A', 'd54ea4a6-0b9c-4e47-bed0-289ae9ff4037', 'Los Chichos', 191.68608, 0.561589945857, 0.420570307208, 1984)
('TRACCSW128F148C7C3', 'If I...', 'SODMVJR12A6D4F985D', 'Ill Na Na', 'ARDI88R1187B98DAB2', 'fd87374e-ffde-4d36-89a8-8a073f795666', 'Foxy Brown', 222.92853, 0.687687485872, 0.406686099467, 0)
('TRACCXJ128F428F0CF', "Werther - Lyric Drama in four Acts/Act I/Alors_ c'est bien ici la maison du bailli?", 'SOIWBDR12A8C13A4AC', 'Massenet: Werther', 'ARUKJUP12086C14589', '8a5f2736-bcde-4a2e-8d50-72631d66a7ef', 'Ramón Vargas;Vladimir Jurowski', 278.38649, 0.391741394148, 0.291264811753, 0)
('TRACCVS12903D014F8', 'Ad Te Clamamus 

In [22]:


conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
# we build the SQL query
q = "PRAGMA table_info(songs);"
# we query the database
t1 = time.time()
res = conn.execute(q)
first_track_meta_cols = res.fetchall()
t2 = time.time()
print('all artist names extracted (SQLite) in:',strtimedelta(t1,t2))
# we close the connection to the database
conn.close()
# let's see some of the content
for s in first_track_meta_cols:
    print(s)

all artist names extracted (SQLite) in: 0:00:00.000454
(0, 'track_id', 'text', 0, None, 1)
(1, 'title', 'text', 0, None, 0)
(2, 'song_id', 'text', 0, None, 0)
(3, 'release', 'text', 0, None, 0)
(4, 'artist_id', 'text', 0, None, 0)
(5, 'artist_mbid', 'text', 0, None, 0)
(6, 'artist_name', 'text', 0, None, 0)
(7, 'duration', 'real', 0, None, 0)
(8, 'artist_familiarity', 'real', 0, None, 0)
(9, 'artist_hotttnesss', 'real', 0, None, 0)
(10, 'year', 'int', 0, None, 0)
